In [1]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.ml.functions import vector_to_array
from pyspark.sql import functions as F
import argparse
from datetime import datetime

/opt/conda/miniconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
table = 'reddit_table'
model_path = '/user/bda_reddit_pw/models/sentiment_model'
result_path = '/user/bda_reddit_pw/crypto_sentiment/csv_data'
day_shift = 37

spark = SparkSession.builder \
    .master('local[1]') \
    .appName('Sentiment Prediction') \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

In [3]:
df = spark.sql(f'SELECT * FROM crypto_sentiment_train_table')
df.show()

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


+-------------------+---------+------------------+--------------+
|               date|sentiment|             price|cryptocurrency|
+-------------------+---------+------------------+--------------+
|2022-11-07 00:00:00|      1.0|0.4034156304688636|       cardano|
|2022-11-07 00:00:00|      1.0|20921.822265010724|       bitcoin|
|2022-11-07 01:00:00|      1.0|0.1151208526143615|      dogecoin|
|2022-11-07 02:00:00|      1.0| 20959.77981937212|       bitcoin|
|2022-11-07 03:00:00|      1.0|21011.162050344992|       bitcoin|
|2022-11-07 03:00:00|      1.0|1592.2925157312254|      ethereum|
|2022-11-07 04:00:00|      1.0| 20927.63993452077|       bitcoin|
|2022-11-07 04:00:00|      0.0|0.4737224095419358|           xrp|
|2022-11-07 05:00:00|      0.0|0.4080714038447476|       cardano|
|2022-11-07 05:00:00|      0.5| 20887.68328944669|       bitcoin|
|2022-11-07 08:00:00|      0.0| 20715.97296960455|       bitcoin|
|2022-11-07 09:00:00|      0.0| 20709.68707916784|       bitcoin|
|2022-11-0

In [4]:
loaded_model = PipelineModel.load(model_path)

In [5]:
test_df = spark.sql(f'SELECT * FROM {table}')

In [6]:
test_df = test_df.withColumnRenamed('selftext', 'text')

In [7]:
test_df.show()

+--------------------+------------+--------------+----+-----+---+----+
|                text| created_utc|cryptocurrency|year|month|day|hour|
+--------------------+------------+--------------+----+-----+---+----+
|disclaimer long b...|1.66458483E9|       cardano|2022|   10|  1|   0|
|bob send address ...|1.66458573E9|       bitcoin|2022|   10|  1|   0|
|back get much res...|1.66458944E9|      dogecoin|2022|   10|  1|   1|
|glaring lack lite...| 1.6645897E9|       bitcoin|2022|   10|  1|   2|
|sitting wallet bl...| 1.6645952E9|       bitcoin|2022|   10|  1|   3|
|huge advocate thi...|1.66459558E9|      ethereum|2022|   10|  1|   3|
|know time chain w...|1.66459661E9|       bitcoin|2022|   10|  1|   3|
|discord invite li...|1.66459686E9|           xrp|2022|   10|  1|   4|
|      safe left know|1.66459994E9|       bitcoin|2022|   10|  1|   4|
|please utilize st...|1.66460083E9|       bitcoin|2022|   10|  1|   5|
|see lot people ha...|1.66460147E9|       bitcoin|2022|   10|  1|   5|
|hello

In [8]:
prediction_raw = loaded_model.transform(test_df)
prediction = (prediction_raw.withColumn("xs", vector_to_array("probability"))).select(['prediction'] + [F.col("xs")[1]] + ['created_utc'] + ['cryptocurrency'])
prediction = prediction.withColumnRenamed('xs[1]', 'probability')
prediction = prediction.withColumn('created_utc', F.date_format(prediction.created_utc.cast('timestamp'), "yyyy-MM-dd HH:00:00"))
prediction.show()

+----------+--------------------+-------------------+--------------+
|prediction|         probability|        created_utc|cryptocurrency|
+----------+--------------------+-------------------+--------------+
|       1.0|   0.999265107599642|2022-10-01 00:00:00|       cardano|
|       1.0|  0.5713289798314397|2022-10-01 00:00:00|       bitcoin|
|       1.0|  0.9999991498271465|2022-10-01 01:00:00|      dogecoin|
|       1.0|  0.9998426257590661|2022-10-01 02:00:00|       bitcoin|
|       1.0|  0.9999994678764528|2022-10-01 03:00:00|       bitcoin|
|       1.0|   0.999999989582975|2022-10-01 03:00:00|      ethereum|
|       1.0|  0.9977463082172485|2022-10-01 03:00:00|       bitcoin|
|       0.0|3.685617606663527E-7|2022-10-01 04:00:00|           xrp|
|       1.0|  0.9994964812952687|2022-10-01 04:00:00|       bitcoin|
|       0.0|2.747458827112581E-8|2022-10-01 05:00:00|       bitcoin|
|       1.0|                 1.0|2022-10-01 05:00:00|       bitcoin|
|       0.0|1.730676131295183...|2

In [9]:
prediction.toPandas()

,prediction,probability,created_utc,cryptocurrency
0,1.0,9.992651e-01,2022-10-01 00:00:00,cardano
1,1.0,5.713290e-01,2022-10-01 00:00:00,bitcoin
2,1.0,9.999991e-01,2022-10-01 01:00:00,dogecoin
3,1.0,9.998426e-01,2022-10-01 02:00:00,bitcoin
4,1.0,9.999995e-01,2022-10-01 03:00:00,bitcoin
...,...,...,...,...
7019,0.0,8.141710e-12,2022-12-31 21:00:00,bitcoin
7020,1.0,9.572387e-01,2022-12-31 22:00:00,solana
7021,1.0,1.000000e+00,2022-12-31 22:00:00,bitcoin
7022,1.0,9.998590e-01,2022-12-31 23:00:00,bitcoin


In [10]:
aggregation = prediction.groupBy(['created_utc', 'cryptocurrency']).agg(F.avg('probability'))
aggregation = aggregation.withColumnRenamed('avg(probability)', 'sentiment')
aggregation = aggregation.withColumnRenamed('cryptocurrency', 'cryptocurrency_a')
## version with prediction 0 1 not probability
# results = prediction.groupBy('created_utc').agg(F.avg('prediction'))
# results = results.withColumnRenamed('avg(prediction)', 'avg_prediction')
aggregation = aggregation.withColumn("sub_utc", F.col("created_utc") + F.expr(f'INTERVAL {day_shift} DAYS'))
aggregation.show()

+-------------------+----------------+--------------------+-------------------+
|        created_utc|cryptocurrency_a|           sentiment|            sub_utc|
+-------------------+----------------+--------------------+-------------------+
|2022-10-02 15:00:00|          solana|3.176570032403791...|2022-11-08 15:00:00|
|2022-10-03 05:00:00|         cardano|1.730676131295183...|2022-11-09 05:00:00|
|2022-10-03 13:00:00|        ethereum|  0.9999789907584345|2022-11-09 13:00:00|
|2022-10-04 16:00:00|        ethereum|  0.9999977756404372|2022-11-10 16:00:00|
|2022-10-02 16:00:00|         cardano|  0.9999999997260065|2022-11-08 16:00:00|
|2022-10-01 13:00:00|        dogecoin|6.622924431098909...|2022-11-07 13:00:00|
|2022-10-02 14:00:00|         cardano| 0.43076400326133435|2022-11-08 14:00:00|
|2022-10-01 03:00:00|         bitcoin|  0.9988728880468507|2022-11-07 03:00:00|
|2022-10-03 22:00:00|         bitcoin| 0.13288958876996504|2022-11-09 22:00:00|
|2022-10-05 13:00:00|        ethereum|4.

In [11]:
crypto_df = spark.sql(f'SELECT * FROM crypto_table')
crypto_df = crypto_df.withColumn('date', F.date_format(crypto_df.date.cast('timestamp'), "yyyy-MM-dd HH:00:00"))
crypto_df.show()

+-------------------+------------------+--------------+----+-----+---+----+
|               date|             price|cryptocurrency|year|month|day|hour|
+-------------------+------------------+--------------+----+-----+---+----+
|2022-09-30 20:00:00| 19674.48240468394|       bitcoin|2022|    9| 30|  20|
|2022-09-30 21:00:00|19416.679427599574|       bitcoin|2022|    9| 30|  21|
|2022-09-30 22:00:00|19390.772416700056|       bitcoin|2022|    9| 30|  22|
|2022-09-30 23:00:00|19391.879627904444|       bitcoin|2022|    9| 30|  23|
|2022-10-01 00:00:00| 19476.92659600407|       bitcoin|2022|   10|  1|   0|
|2022-10-01 01:00:00| 19408.74499886766|       bitcoin|2022|   10|  1|   1|
|2022-10-01 02:00:00|19399.760776852163|       bitcoin|2022|   10|  1|   2|
|2022-10-01 03:00:00| 19416.21089144999|       bitcoin|2022|   10|  1|   3|
|2022-10-01 04:00:00| 19403.31901570052|       bitcoin|2022|   10|  1|   4|
|2022-10-01 05:00:00| 19400.51033859877|       bitcoin|2022|   10|  1|   5|
|2022-10-01 

In [16]:
results = aggregation.join(crypto_df, (aggregation.sub_utc ==  crypto_df.date) & (aggregation.cryptocurrency_a ==  crypto_df.cryptocurrency),"right")
results = results.select('date', 'sentiment', 'price', 'cryptocurrency')

In [20]:
res = results.toPandas()
res = res.interpolate(method='linear').dropna()
results = spark.createDataFrame(res)

,date,sentiment,price,cryptocurrency
882,2022-11-07 00:00:00,0.571329,20921.822265,bitcoin
883,2022-11-07 01:00:00,0.785586,20883.301226,bitcoin
884,2022-11-07 02:00:00,0.999843,20959.779819,bitcoin
885,2022-11-07 03:00:00,0.998873,21011.162050,bitcoin
886,2022-11-07 04:00:00,0.999496,20927.639935,bitcoin
...,...,...,...,...
16237,2023-01-21 16:00:00,0.986978,25.327134,solana
16238,2023-01-21 17:00:00,0.997168,25.576211,solana
16239,2023-01-21 18:00:00,0.998112,25.477412,solana
16240,2023-01-21 19:00:00,0.999056,25.459143,solana


In [ ]:
results = results.sort(F.col("date"))

In [ ]:
test_data = aggregation.filter(F.col("sub_utc") > F.lit(datetime.now().isoformat()))
test_data = test_data.select('sub_utc', 'sentiment', 'cryptocurrency_a')
test_data = test_data.withColumnRenamed('sub_utc', 'date')
test_data = test_data.withColumnRenamed('cryptocurrency_a', 'cryptocurrency')
test_data = test_data.sort(F.col("date"))
test_data.tail(10)

In [ ]:
results.toPandas()

In [ ]:
cmd = f'hdfs dfs -rmr {result_path}'
os.system(cmd)
results.write.csv(result_path)